# Database Systems 4020

## Riad Mohammed

### Activity 3

### Question 1

#### (a) (5 points) Execute a single query to return and print out the total assets owned by the bank ( i.e., a cumulative total of the assets at all branches)

In [147]:
import mysql.connector
host = 'bankdb2.cvh022knj1xf.us-east-1.rds.amazonaws.com'
user = 'admin'
password = "Kisscake12!"
conn = mysql.connector.connect(host=host,user=user,password=password)

cursor=conn.cursor()
cursor.execute('USE bank')
query="select sum(assets)from branch;"
cursor.execute(query)

total = cursor.fetchone()[0]
print(f'\n\nTotal assets owned by the bank: ${total:,.2f}')
conn.close()



Total assets owned by the bank: $24,600,479.98


#### (b) (10 points) Create a trigger named maintain_branch_size on your branch table that ensures that a small branch does not become a large branch (i.e., that its total assets never gets to 3 million dollars). Whenever an update to the assets of a small branch is attempted that would cause it to become a large branch, your trigger should store this excess money in a table (e.g. bailout_cache) to be later divided equally among the existing large branches, capping the small branch at 2,999,999 in total assets.


In [148]:
import mysql.connector
host = 'bankdb2.cvh022knj1xf.us-east-1.rds.amazonaws.com'
user = 'admin'
password = "Kisscake12!"
conn = mysql.connector.connect(host=host,user=user,password=password)

cursor=conn.cursor()
cursor.execute('USE bank')

query="CREATE TABLE bailout_cache(amount NUMERIC(12,2) DEFAULT 0);"
cursor.execute(query)
conn.close()

In [149]:
import mysql.connector
host = 'bankdb2.cvh022knj1xf.us-east-1.rds.amazonaws.com'
user = 'admin'
password = "Kisscake12!"
conn = mysql.connector.connect(host=host,user=user,password=password)

cursor=conn.cursor()
cursor.execute('USE bank')

query="INSERT INTO bailout_cache(amount) values (0);"
cursor.execute(query)
conn.commit()
conn.close()

In [150]:
import mysql.connector
host = 'bankdb2.cvh022knj1xf.us-east-1.rds.amazonaws.com'
user = 'admin'
password = "Kisscake12!"
conn = mysql.connector.connect(host=host,user=user,password=password)

cursor=conn.cursor()
cursor.execute('USE bank')




query="""
         CREATE TRIGGER maintain_branch_size BEFORE UPDATE ON branch FOR EACH ROW
         BEGIN
            IF NEW.assets >=3000000 AND OLD.assets < 3000000 THEN
               
               UPDATE bailout_cache set amount=amount+(NEW.assets-2999999.99);
               SET NEW.assets=2999999.99;
            END IF;
         END;"""
    
cursor.execute(query)
conn.commit()
conn.close()
    
    
    #INSERT INTO NEW.assets(2999999.99)
    #DECLARE large_branch_flag NUMERIC(12,2) DEFAULT 2999999.99
    # DECLARE bailout_cash NUMERIC(12,2)
    #DELETE FROM assets WHERE amount=assets
    


#### (c) (5 points) Execute a query that attempts to divide the bailout money among the small branches as a test of your trigger.


In [151]:
import mysql.connector
host = 'bankdb2.cvh022knj1xf.us-east-1.rds.amazonaws.com'
user = 'admin'
password = "Kisscake12!"
conn = mysql.connector.connect(host=host,user=user,password=password)
cursor=conn.cursor()
cursor.execute('USE bank')

query="select count(branch_name)from branch where (assets<3000000)"
cursor.execute(query)
amount_small=cursor.fetchone()[0]
#print(amount_small)

query="select branch.branch_name,branch.branch_city,branch.assets from branch ORDER BY assets DESC"

cursor.execute(query)
results=cursor.fetchall()

print ("{:<20} {:<20} {:<20}".format('branch_name','branch_city','assets'))
for name in results:
    branch_name,branch_city,assets=name
    print ("{:<20} {:<20} {:<20}".format(branch_name,branch_city,assets))

query="update branch set assets=assets + %s where assets<3000000"%(10000000/amount_small)
cursor.execute(query)
conn.commit()

amount_liq = cursor.fetchall()
#print(amount_liq)


conn.close()

branch_name          branch_city          assets              
Round Hill           Horseneck            7000000.00          
Brighton             Brooklyn             6125000.00          
North Town           Rye                  3237500.00          
Redwood              Palo Alto            2489583.33          
Perryridge           Horseneck            2089583.33          
Downtown             Brooklyn             1289583.33          
Central              Rye                  789863.33           
Mianus               Horseneck            789783.33           
Pownal               Bennington           789583.33           


Above shows the branches before bailout money was allocated to small branches

In [152]:
import mysql.connector
host = 'bankdb2.cvh022knj1xf.us-east-1.rds.amazonaws.com'
user = 'admin'
password = "Kisscake12!"
conn = mysql.connector.connect(host=host,user=user,password=password)
cursor=conn.cursor()
cursor.execute('USE bank')

query="select branch.branch_name,branch.branch_city,branch.assets from branch ORDER BY assets DESC"

cursor.execute(query)
results=cursor.fetchall()

print ("{:<20} {:<20} {:<20}".format('branch_name','branch_city','assets'))
for name in results:
    branch_name,branch_city,assets=name
    print ("{:<20} {:<20} {:<20}".format(branch_name,branch_city,assets))

branch_name          branch_city          assets              
Round Hill           Horseneck            7000000.00          
Brighton             Brooklyn             6125000.00          
North Town           Rye                  3237500.00          
Perryridge           Horseneck            2999999.99          
Redwood              Palo Alto            2999999.99          
Downtown             Brooklyn             2956250.00          
Central              Rye                  2456530.00          
Mianus               Horseneck            2456450.00          
Pownal               Bennington           2456250.00          


Above shows the branches after bailout money was allocated to small branches

In [153]:
import mysql.connector
host = 'bankdb2.cvh022knj1xf.us-east-1.rds.amazonaws.com'
user = 'admin'
password = "Kisscake12!"
conn = mysql.connector.connect(host=host,user=user,password=password)

cursor=conn.cursor()
cursor.execute('USE bank')
query="select amount from bailout_cache;"
cursor.execute(query)

bailout_cash = cursor.fetchone()[0]
print(f'Bailout_cash left: ${bailout_cash}')

Bailout_cash left: $1912500.02


#### (d) (3 points) Execute a query that divides any excess money from the bailout (money in your bailout_cache) among the large branches


In [154]:
import mysql.connector
host = 'bankdb2.cvh022knj1xf.us-east-1.rds.amazonaws.com'
user = 'admin'
password = "Kisscake12!"
conn = mysql.connector.connect(host=host,user=user,password=password)
cursor=conn.cursor()
cursor.execute('USE bank')

query="select count(branch_name)from branch where (assets>=3000000)"
cursor.execute(query)
amount_large=cursor.fetchone()[0]
print(f'Number of Large Companies: {amount_large}')
print(f'\n\nBailout_cash left: ${bailout_cash:,.2f}')

query="select branch.branch_name,branch.branch_city,branch.assets from branch ORDER BY assets DESC"

cursor.execute(query)
results=cursor.fetchall()

print ("{:<20} {:<20} {:<20}".format('branch_name','branch_city','assets'))
for name in results:
    branch_name,branch_city,assets=name
    print ("{:<20} {:<20} {:<20}".format(branch_name,branch_city,assets))

query="update branch set assets=assets + %s where assets>=3000000"%(bailout_cash/amount_large)
cursor.execute(query)
conn.commit()

##amount_liq = cursor.fetchall()





conn.close()

Number of Large Companies: 3


Bailout_cash left: $1,912,500.02
branch_name          branch_city          assets              
Round Hill           Horseneck            7000000.00          
Brighton             Brooklyn             6125000.00          
North Town           Rye                  3237500.00          
Perryridge           Horseneck            2999999.99          
Redwood              Palo Alto            2999999.99          
Downtown             Brooklyn             2956250.00          
Central              Rye                  2456530.00          
Mianus               Horseneck            2456450.00          
Pownal               Bennington           2456250.00          


Above shows the branches before excess bailout money was allocated to large branches

In [155]:
import mysql.connector
host = 'bankdb2.cvh022knj1xf.us-east-1.rds.amazonaws.com'
user = 'admin'
password = "Kisscake12!"
conn = mysql.connector.connect(host=host,user=user,password=password)
cursor=conn.cursor()
cursor.execute('USE bank')

query="select branch.branch_name,branch.branch_city,branch.assets from branch ORDER BY assets DESC"

cursor.execute(query)
results=cursor.fetchall()

print ("{:<20} {:<20} {:<20}".format('branch_name','branch_city','assets'))
for name in results:
    branch_name,branch_city,assets=name
    print ("{:<20} {:<20} {:<20}".format(branch_name,branch_city,assets))

branch_name          branch_city          assets              
Round Hill           Horseneck            7637500.01          
Brighton             Brooklyn             6762500.01          
North Town           Rye                  3875000.01          
Perryridge           Horseneck            2999999.99          
Redwood              Palo Alto            2999999.99          
Downtown             Brooklyn             2956250.00          
Central              Rye                  2456530.00          
Mianus               Horseneck            2456450.00          
Pownal               Bennington           2456250.00          


Above shows the branches after excess bailout money was allocated to large branches

#### (e) (2 points) Execute a single query to return and print out the total assets owned by the bank ( i.e., a cumulative total of the assets at all branches) after all changes above has been made.


In [156]:
import mysql.connector
host = 'bankdb2.cvh022knj1xf.us-east-1.rds.amazonaws.com'
user = 'admin'
password = "Kisscake12!"
conn = mysql.connector.connect(host=host,user=user,password=password)

cursor=conn.cursor()
cursor.execute('USE bank')
query="select sum(assets)from branch;"
cursor.execute(query)

total = cursor.fetchone()[0]
print(f'\n\nTotal assets owned by the bank after changes: ${total:,.2f}')
conn.close()



Total assets owned by the bank after changes: $34,600,480.01
